In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# pip install langchain langchain-openai langchain-community langchain-core youtube-transcript-api faiss_cpu 

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from youtube_transcript_api.proxies import WebshareProxyConfig
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import YoutubeLoader

# 1. Indexing

# 1.1 Document Ingestion

In [35]:
try:
    # using webshare proxy to fetch youtube transcripts


    video_id = 'nlz9j-r0U9U'
    
    ytt_api = YouTubeTranscriptApi(
        proxy_config=WebshareProxyConfig(
            proxy_username="",
            proxy_password="",
        )
    )

    
    # all requests done by ytt_api will now be proxied through Webshare
    ytt_transcripts = ytt_api.fetch(video_id, languages=['en']) # can also use 'hi' for hindi


except TranscriptsDisabled:
    print("Transcripts are disabled for this video.")
except Exception as e:
    print(f"Error: {e}")

Error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=nlz9j-r0U9U! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (nlz9j-r0U9U) transcripts are available in the following languages:

(MANUALLY CREATED)
None

(GENERATED)
 - hi ("Hindi (auto-generated)")

(TRANSLATION LANGUAGES)
None

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!


In [75]:
# combined transcript
combined_transcript = "".join(res.text for res in ytt_transcripts)



In [6]:
# workwound of youtube transcript

# combined_transcript = """If you're using Chhatubt, Claude,Gemini, Perplexity, or other AI toolsand you're getting responses that soundconfident but might be completely wrong,you're probably thinking, "Well, I justneed to be more specific, or maybe Ineed better AI tools." That's exactlywhat I thought, too. But after testingthousands of prompts, I discoveredsomething that completely shatteredeverything I believed about AI. Theproblem is that most people are speakinga completely different language thanwhat AI actually understands. The skillof bridging that language gap is calledprompt engineering. So, in this video,I'll teach you how to speak AI's actuallanguage. Plus, I'll show you advancedhacks that make AI admit when it'sguessing, get AI to quality controlitself, and give you way more confidencein AI's output. All right, let me showyou what's actually going on behind thescenes when you get inconsistent AIresults. AI seems unpredictable, butthere's actually a method to themadness. Every response follows exactmathematical patterns based on what'scalled token probability. So when youtype write me an email, the AIcalculates the probability of whatshould come next. But most people aregiving it the wrong type of data. You'regiving it random words when AI needsstructured probability inputs. Withoutthe right type of data, the AI isspinning a roulette wheel betweenmillions of different email patterns.Business email, personal email, salesemail, breakup email. It's literallyguessing. But once you understand how tofeed it structured data inputs insteadof random details, everything changes.You guide it to the exact pattern thatyou want instead of letting it guess.And that's exactly what I'm about toshow you. Now that you understand how AIactually works, and before I show youthe crazy hacks that are going to blowyour mind, you need to understand thefoundation that makes everything elsework. The gold standard for promptingand what gets consistently amazingresults is using a six-part framework.This aligns with what Google teaches intheir 9-hour prompt engineering coursethat I completed. This isn't a secret.It's just that most people never learnthe structure. Let me walk you throughthis using a quick example. Let's sayyou have a fitness app and you'recrafting an email to reach out to yourcustomers. Number one is the role. Tellthe AI who it is. This sets thefoundation for how AI thinks andresponds. So, you tell the AI who itshould be. Instead of getting genericrobotic answers, this gives you aspecific voice and expertise. So in thisexample, we'd start with you are afitness app founder. Number two is thecontext. So set the situation and givethe AI some background. So for thisexample, we could say reaching out tousers who downloaded the app 2 weeks agobut haven't logged a single workout yet.So just explaining the background.Number three is the task. So beingspecific about what we actually wantedto do. So, for this example, we couldsay, "Write an encouraging email thatmotivates them to try their first10-minute workout without making themfeel guilty." Number four is the format.So, we need to define the output. So, dowe want it in bullet point form? Do wewant a certain amount of word count? Weneed to specify how we want theinformation. So for this example, wecould say something like return as a100word email with an upbeat subjectline, empathetic opening, one specificaction step, and motivational closingtells exactly what we're looking for.Number five is rules. So set someboundaries or constraints. Is thereanything you don't want it to talkabout? Are there things that you do wantit to say specifically? This is theopportunity to do that. So for thisexample, we could say something like, dokeep it supportive and encouraging.Don't use guilt, shame, or pressuretactics. Tells it exactly what to avoid.And number six is examples. This is thesecret sauce and where most people missthe real power. You can show the AI whatgood looks like and you can do it in acouple of different ways. Depending onwhat you're actually looking for AI tohelp you with, you can give it textexamples or you can upload actualimages, emails, documents for the AI toanalyze and match. So, for our fitnessapp example, I could upload amotivational email and say somethinglike this. Match this energetic andsupportive tone that makes people feelcapable and referred to the uploadedimage. Now, let me show you thetransformation that this makes in realtime. Here's what a lot of people wouldtype. Write me a fitness marketing emailfor my fitness app. And here's what weget. Generic, templated, could be forany business. Now, let's put all sixparts together of the example that wejust went through just to compare. Youare a fitness app founder, reaching outto users who downloaded your app twoweeks ago but haven't logged a singleworkout. Write an encouraging email thatmotivates them to try their 10-minuteworkout without making them feel guilty.Return as a 100word email with an upbeatsubject line, empathetic opening, onespecific action step, motivationalclosing. Do keep it supportive andencouraging. Don't use guilt, shame, orpressure tactics. And match thisenergetic, supportive tone that makespeople feel capable. And refer to theuploaded image. Look at thattransformation. It sounds like it'swritten by a real person and speaks tothe exact issues that we're trying toaddress. This foundation makeseverything else I'm about to show youwork 10 times better. All right, let meshow you the six gamechanging hacks thatwill take this foundation and turn itinto something incredible. Hack numberone is the truth detector. This is thehack that prevents embarrassing AImistakes that could damage yourreputation or cost you money. Mostpeople don't realize that AI soundsconfident even when it's completelyguessing. So, I force it to rate its ownconfidence. I add this to everyimportant prompt. For each claim, rateyour confidence. Virtually certain, 95%and above. Highly confidence, 80 to 95%.Moderately confident, 60 to 80%,speculative, or low confidence, below40%. And explain your confidence level.Let me show you this in action. I'mgoing to ask it something it may not beentirely sure about. Let me use theframework we just learned. You are amarketing consultant with 10 years ofexperience in restaurant marketing.That's our role. Client is asking aboutstrategies to increase foot traffic andonline orders for their restaurant.That's the context. Provide fivestrategies with explanations. That's thetask. Return as a numbered list withbrief explanations for each strategy.That's the format we're looking for. Dofocus on proven methods and emergingtrends. Don't include outdated tactics.That's the rule. That's our constraint.And for each strategy, rate yourconfidence. This is where I'm going toput in what we just talked about.virtually certain, highly confident,etc. And let's see what it comes backwith. See how it's telling me it'svirtually certain about strategy numberone. Then it moves to highly confidentfor strategy number two, and then downto the last couple, four and five. It'sonly moderately confident in those ones.Now, the AI can tell me when it'sbasically guessing. This has saved mefrom so many embarrassing mistakes. Nomore blindly trusting AI when it's justmaking educated guesses. Now that youknow how to catch AI when it's makingthings up, what I'm about to show youwill completely change how you createprompts in the first place. Hack numbertwo is the AI prompt helper. This issuch a game changer. Another method thatI use when I'm struggling with promptsis I get AI to help me with theprompting itself. There's two awesomeways to do this. Approach number one isstarting from scratch. Instead of tryingto figure out the best way to ask forsomething with a prompt, if I waslooking to use AI to help plan a weekendtrip, I could simply say, "I want toplan a perfect weekend getaway for twopeople on an $800 budget within drivingdistance of Chicago. Write the optimalprompt that I should use to get the besttravel recommendations from you." Lookat this. The AI just wrote me a nicelystructured prompt with context, specificinstructions, the output format, and itknows how it wants to be prompted. Andnow I'm going to use the prompt that itjust gave me. I'm going to copy andpaste that right into the prompt box.And here's what we get. Amazing, right?It's given us some really goodrecommendations in here with things todo, where to stay, the cost breakdown,how long the driving distances away fromChicago. Perfect. This is great. Now,here's approach number two, and thisone's for when you're frustrated withyour AI results. Let's say you alreadywrote a prompt, but you gotdisappointing results. Instead ofgetting frustrated and trying to guesswhat went wrong, I'm going to say, Itried this prompt. Plan a weekend tripfor me and my friend. We have about $800to spend and want to go somewhere withindriving distance. Give me some ideas.But the output wasn't great. Can youanalyze my original prompt and improveit to get a better result? And look whatit generated. It's identifying exactlywhat was missing. Too vague on location,no interest mentioned, no context ontravel dates, no preference on the vibe,and then it gives us an improved versionof the prompt where we can fill inadditional details that we were missing.Whether you're starting from scratch orfixing what you already have, AI becomesyour prompting partner. I use this allthe time for everything from personalstuff, business strategy, contentcreation, and even data analysis.Instead of trying to guess why I didn'tget the best result from a prompt, Ijust ask AI for feedback on the promptitself. But if you thought that havingAI write and improve your prompts wasnext level, then wait until you see whathappens when you stop using the same AImodel for everything. Hack number three.Here's what separates the AI beginnersfrom the pros. Your AI tool of choicehas different models that excel atdifferent things. And the model that youchoose can significantly impact thequality of your output and your results.The secret is understanding which modeldoes the task that you're asking it todo the best. Let me show you what I meanusing chat GBT as an example. So, if welook at ChatGpt, it has a bunch ofdifferent models. The 40 is great foreveryday tasks. The 4.5 excels at tasksrequiring a lot of emotionalintelligence, like creative writing, forexample. The 03 and the O4 Mini arebuilt for more deeper reasoning andcomplex problem solving. The 4.1 isgreat for analyzing information andgreat for business strategy. And the 4.1Mini is perfect for just quick,straightforward requests. Let me showyou. So, I'm going to give it a promptusing our six-part framework with theGPT 4.5 model for a creative writingtask. Here's the prompt I'm going togive it. You're a published novelistknown for emotionally resonant literaryfiction. A reader has just found an oldyellow tucked letter behind a loosefloorboard in their grandmother's attic,etc., etc., and write opening paragraphsthat capture the moments of thediscovery. Don't rush to reveal what theletter says, example tone, and it's verydescriptive kind of language. So, at theend of the prompt, we give it an exampleof what we're looking for in terms ofthe quality of the writing. All right,we're going to generate and see what itcomes up with. Wow, look at the result.You could see it's very descriptivelanguage and it really captured theexample that we gave it really well atthe end of our prompt, the example tone,and it really does sound like somethingyou'd read in a novel. Now, I'm going touse the exact same prompt with the 40model, which is more for kind ofeveryday uses, multiple different typesof uses, and see the comparison betweenthe two. Now, this one's pretty good,too, but when you look at the words it'susing, it's not nearly as descriptive ordramatic as what we got from the 4.5model. So the key takeaway here is thatthe model that you select can have asignificant impact on the result thatyou get. And the same principle appliesto whatever AI you're using, whetherit's Claude, Perplexity, Gemini, orother AI tools. They all have differentmodels optimized for different tasks.You wouldn't use a chef knife to flippancakes, right? Using the right modelfor the right job makes a hugedifference. Now, that hack was powerful,but if you don't combine it with whatI'm about to show you next, the way tomake AI its own editor, then you'remissing the real magic. Hack numberfour. This one's my favorite, andhonestly, it's like discovering that AIhas a hidden superpower. I accidentallydiscovered this when I got frustratedwith bad output. Instead of startingover, I asked AI to critique its ownwork. Let me show you this in action.I'm going to start with a LinkedIn postrequest using the framework that wetalked about. You are a career coachhelping professionals advance in theircareer. A mid-level professional intheir 30s wants to share insights aboutnetworking to build their personal brandon LinkedIn. Write a LinkedIn post aboutthe importance of networking for careergrowth. Return it as a 150word post withan engaging hook, a couple actionableinsights, questions to drive engagement,make it personal and authentic. Don'tuse corporate jargon or cliches and Igive it an example tone and we're goingto see what it comes up with. Okay,great. Here's the first draft. It'spretty good actually because we use theframework and it's structured, has theright tone, includes actionable advice.But watch what happens when I use theself-improvement loop. Now, I'm going tosay analyze your previous response andidentify three specific weaknesses. Thenrewrite it addressing those issues. Dothis three times, focusing on differentaspects each round. Look at this. It'sidentifying its own weaknesses. Thefirst draft, surface levelvulnerability, lack of emotional stakes,missed opportunities to reflect on theoutcome. And then here's version two.Much better, right? Looks better. Toneseems to be better. for improvedstorytelling and emotional engagement.But we're not done. Now, here's theoutput from round three. This one wasfocused on the call to action andengagement. The AI basically becomes itsown editor. Look at the progression. Wewent from good to great. Now, we have abunch of different versions that we canreview and decide which one we likebest. Or we can just ask AI whichversion is the best, like this. In thiscase, it liked round two overall becauseof the storytelling and therelatability, but it also liked round3's call to action. So, at the end ofthe output, it actually asks us if wewant to create a hybrid version thatcombines the strengths of the round twoand three versions, which I I think is agreat approach. So, instead of justaccepting the first version's output,you're forcing it to keep improving.It's like having a built-in qualitycontrol team that never gets tired. Now,what if I told you that there's foursimple words that can significantlyincrease the accuracy of any strategicor complex task. Hack number five is thefour-word miracle. Add these four wordsto any strategic or complex prompt.Think step by step. That's it. But don'tunderestimate it. I've tested this outon a lot of prompts and it consistentlygives you better, clearer, more reliableresults, especially for things likebusiness planning, marketing strategy,and content creation. Why? becauseyou're asking it to show its thoughtprocess instead of just jumping straightto the answer. Let me show you. Here's astrategic planning request. I want tocreate a content strategy for my fitnesscoaching business. I post on Instagramthree times per week. I have 5,000followers and I want to increaseengagement. My audience is busyprofessionals 28 to 40. Create a 30-daycontent plan. Now, when it generates theresponse, it gives me recommendations,but I don't see the thinking behind themor how everything connects together.Now, let me add those four magic words,think step by step, and we'll see whatit comes up with. Wow. See, look at thisbig difference. It's showing me thestrategic thinking step by step, theaudience analysis, content pillars,posting schedule, engagement tactics.Each piece builds on the last, and I canactually see the thinking behind everyrecommendation. Now, some of you mightbe wondering, well, why not use the 03model? Doesn't it show its workautomatically? And that's a greatquestion because yes, there are moreadvanced models like the O3 model thatdo often provide built-in reasoning. Buthere's the thing, not everyone hasaccess to those models. And for taskslike content strategy, like the examplethat we just went through, they're oftenoverkill based on my experience. So ifyou're using the free version ofChatubt, or you just want to keep thingssimple, this trick gives you alightweight workaround for better, morestrategic results. This can work forbusiness planning, calculations,decision-making, anything where you wantclearer thinking without the complexity.That four-word trick is incrediblypowerful for reliability, but this nexthack is my secret weapon for gettingstrategic answers you'd normally have topay a consultant for. Hack number six isthe priming trick. Instead of jumpingstraight to my specific question, I'mgoing to ask a broader question first toactivate all the AI's relevantknowledge. Let's say I was creating anonline Excel course for adults and Iwanted some help with the coursestructure and the program. First, I'mgoing to ask it this. What psychologicaland educational factors make onlinelearning most effective for adultlearners? And here's what it generates.Wow, look at this response. Verydetailed. Talks about the autonomy ofthe adults, self-direction, internalmotivations, practical applications,confidence building. Really goodinsights for us to then use for the nextprompt. So, now that we've primed AI'sknowledge and it has all of that greatinformation in its memory, we're goingto follow up with another prompt. Youare an instructional design expertspecializing in adult education. I'mcreating a 2-hour online course aboutExcel for small business owners who areintimidated by spreadsheets and havelimited time. Based on the learningprinciples you just outlined, providesome key strategies for my course,content delivery, and examples. Returnas a structured plan with five keyrecommendations. and focus on reducingoverwhelm and increasing confidence,etc., etc. And here's what it comes upwith. Really, really good information.So, it's taking information from theprevious output that we got from it,psychological factors and things likethat, and it's addressing those thingswithin this prompt, which is amazing.Start with a confidence building quickwin, organize content into bite-sizedstandalone mini lessons. Lots of thoseinsights that we got from the previousprompt are included in here. The AIbasically downloaded its knowledge ofthe adult learning psychology first andthen it applied it to my specificsituation. So this approach gives us waymore strategic thinking that considersthe psychological part of it rather thanjust quick tips. And this primingtechnique works for any complex topic,business strategy, technical problems,creative projects, you name it. So nowyou've got these powerful techniques,but here's the trap that catches 90% ofpeople. They get one good result andthey think they're done. Real promptengineering means testing promptsmultiple times, finding patterns in thefailures, iterating and refining untilit's bulletproof. I have a library of myown tested prompts that work everysingle time. It took time to build, butnow I can handle some of my most tedioustasks in seconds. Now, when I need towrite a piece of content or an email, Idon't cross my fingers and hope for thebest. I use a prompt I've testedmultiple times that I know deliversexactly what I need. And for reallyimportant decisions like strategicbusiness choices or anything that couldimpact my reputation, I'll even run thesame prompt across multiple AI tools andhave one of them critique which responseis the best. That's the differencebetween just celebrating single wins andactually starting to build systems. Sohere's what I want you to do right now.Pick one thing that you ask AI to doregularly. Could be emails, creatingcontent, analyzing data, whichever task.Build a prompt using the six-partframework that I showed you. Add in acouple of the hacks I showed you andthen test it a few times. Refine ituntil the output is actually somethingthat you'd use. Do this for just oneprompt and you'll understand why somepeople are getting incredible resultswhile others are struggling with AI andyou'll join that small group of peoplethat actually knows how to leveragethese tools properly. And trust me, onceyou experience the difference, you'llnever go back to basic prompting again.If you enjoyed this video, then pleaseshow me your support by hitting thethumbs up button. It really helps thechannel. And if you want to learn moreabout how you can use AI to level upyour business and your life, then clickthis next video."""



# 1.2 Text Splitting

In [39]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.create_documents([combined_transcript])

In [81]:
print(len(chunks))

# for chunk in chunks:
#     print(chunk.page_content)

79


# 1.3 Embedding

In [11]:
api_key='your_api_key'

In [13]:
embeddings = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-small")

# 1.4 Storage in Vector Store

In [82]:
vector_store = FAISS.from_documents(chunks, embeddings)

#vector_store.index_to_docstore_id

In [16]:
vector_store.get_by_ids(['9e1017d7-c188-44f8-a684-75a1aca081a4'])

[Document(id='9e1017d7-c188-44f8-a684-75a1aca081a4', metadata={}, page_content="hacks I showed you andthen test it a few times. Refine ituntil the output is actually somethingthat you'd use. Do this for just oneprompt and you'll understand why somepeople are getting incredible resultswhile others are struggling with AI andyou'll join that small group of peoplethat actually knows how to leveragethese tools properly. And trust me, onceyou experience the difference, you'llnever go back to basic prompting again.If you enjoyed this video, then pleaseshow me your support by")]

# 2. Retrievel

In [43]:
# using similarity search

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [83]:
#query = "Summarise langchain in 50 words?"
#retriever.invoke(query)

# 3. Augmentation

In [23]:
prompt = PromptTemplate(
    template="""You are a helpful assistant. Help finding the response to the query using the provided context only. If the query cannot be answered based on the context knowledge, respond back No answer in the context.
        context: {context}
        query: {query}
    """,
    input_variables=["context", "query"]
)

In [84]:
query = "Summarise langchain in 50 words?"
retreived_docs = retriever.invoke(query)

In [52]:
context = '\n\n'.join(doc.page_content for doc in retreived_docs)

In [53]:
final_prompt = prompt.invoke({"context":context, "query": query})

# 4. Generation

In [27]:
llm = ChatOpenAI(api_key=api_key, model="gpt-4o-mini", temperature=0.5) # temperature - range of creativity 1- least creative, 0 - most creative

In [54]:
final_response = llm.invoke(final_prompt)

print(final_response.content)

No answer in the context.


# Combining the Chain

In [66]:
from langchain_core.runnables import RunnableParallel, RunnableSequence, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [59]:
def create_context(retreived_docs):
    context = '\n\n'.join(doc.page_content for doc in retreived_docs)
    return context

In [71]:
parallel_chain = RunnableParallel({
    "context": retriever | RunnableLambda(create_context),
    "query": RunnablePassthrough()
})

In [85]:
query = "Summarise langchain in 50 words?"

#parallel_chain.invoke(query)

In [67]:
str_parser = StrOutputParser()

In [73]:
main_chain = parallel_chain | prompt | llm | str_parser

In [74]:
result = main_chain.invoke(query)


print(result)

LangChain is a powerful library for building LLM-powered applications. It simplifies the integration of various components, enabling natural language understanding and efficient query handling. Users can interact with documents, generate summaries, and create practice questions, making it an invaluable tool for developers in machine learning and AI applications.
